In [5]:
from crewai import Agent, Crew, Task, Process
from pydantic import BaseModel, Field
from crewai.tools import BaseTool
from typing import List
from crewai.project import CrewBase, crew, task, agent
from crewai.tools import tool
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

from langchain_community.utilities import GoogleSerperAPIWrapper

from dotenv import load_dotenv
import os

load_dotenv()


True

In [8]:
# @tool
def get_serper_restaurants(query: str,
                            gl: str,
                            k: int=5,
                            type_content: str='places',
                            hl: str='pt', ) -> List[dict]:
                                
    
    """
    Get the multiple search results from Google Serper API and scrap the content
    
    Args:
    - query: query to be searched
    - gl: Country to be searched
    - k: Number of results to be returned
    - type_content: Type of content to be searched
    - hl: Language to be searched
    
    Returns:
    - List of results from the search
    """

    search = GoogleSerperAPIWrapper(gl=gl,
                                    hl=hl,
                                    k=k, 
                                    type=type_content)
    
    # results = [search.results(query) for query in queries]
    results = search.results(query)
    return results

In [11]:
travel_agent = Agent(
    role="Coletor de restaurantes",
    goal="Coletar restaurantes da culinária {cuisine} em {city} e as principais avaliações",
    backstory="Você é um agente de viagens especialista em recomendar os restaurantes da culinária {cuisine}."
               "Você recomenda aos seus clientes os melhores restaurantes na cidade{city}"
               "E ajuda-los a ter uma experiência gastronômica incrível."
               "Seu trabalho é a base para o sumarizador de restaurantes.",
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    allow_delegation=False,
    verbose=True
)

In [12]:
writer = Agent(
    role="Sumarizar e avaliar restaurantes",
    goal="Escrever um artigo recomendando e sumarizando os restaurantes da culinária {cuisine} em {city}",
    backstory="Você está trabalhando na escrita de "
              "um novo artigo recomendando restaurantes da culinária {cuisine} em {city}. "
              "Você baseia sua escrita no trabalho do "
              "Coletor de restaurantes, que fornece um esboço "
              "e contexto relevante sobre os restaurantes. "
              "Você segue os principais objetivos e a "
              "direção do esboço, conforme fornecido pelo Coletor de restaurantes. "
              "Você também fornece insights objetivos e imparciais "
              "e os fundamenta com as informações "
              "fornecidas pelo Coletor de restaurantes. ",
    allow_delegation=False,
    verbose=True
)

In [22]:
collect_restaurants = Task(
    description=(
        "1. Buscar os MELHORES restaurantes da culinária {cuisine} em {city}.\n"
        "2. Identificar o público-alvo, principais pontos avaliados, melhores pratos e localização\n"
        "3. Desenvolver um esboço detalhado do conteúdo, incluindo "
            "os restaurantes rankeados em ordem descrescente\n"
    ),
    expected_output="Um documento abrangente de restaurantes dividido em tópicos"
        "com o público-alvo, principais pontos avaliados e  melhores pratos",
    agent=travel_agent,
)

In [23]:
write = Task(
    description=(
        "1. Usar o plano de restaurantes para criar um "
            "resumo atraente sobre os restaurantes.\n"
        "2. As seções/subtítulos devem ser nomeados de forma "
            "envolvente.\n"
        "3. Garantir que o post seja estruturado com uma "
            "introdução envolvente, um corpo informativo "
            "e uma conclusão resumida.\n"
        "4. Revisar para corrigir erros gramaticais e "
            "garantir alinhamento.\n"
    ),
    expected_output="Um post de blog bem escrito "
        "em formato markdown, pronto para publicação, "
        "com cada seção contendo 2 ou 3 parágrafos.",
    agent=writer,
)

In [24]:
from crewai import Crew

crew = Crew(
    agents=[travel_agent, writer],
    tasks=[collect_restaurants, write],
    verbose=True
)

2025-01-17 17:50:18,427 - 3888 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [27]:
result = crew.kickoff(inputs={"cuisine": "italiana", "city": "João Pessoa"})

# Agent: Coletor de restaurantes
## Task: 1. Buscar os MELHORES restaurantes da culinária italiana em João Pessoa.
2. Identificar o público-alvo, principais pontos avaliados, melhores pratos e localização
3. Desenvolver um esboço detalhado do conteúdo, incluindo os restaurantes rankeados em ordem descrescente



# Agent: Coletor de restaurantes
## Thought: Eu preciso buscar informações sobre os melhores restaurantes italianos em João Pessoa. Vou começar procurando por uma lista de restaurantes bem avaliados na cidade.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"melhores restaurantes italianos em Jo\\u00e3o Pessoa avalia\\u00e7\\u00f5es\"}"
## Tool Output: 
{'searchParameters': {'q': 'melhores restaurantes italianos em João Pessoa avaliações', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Os 10 melhores restaurantes italianos: João Pessoa - Tripadvisor', 'link': 'https://www.tripadvisor.com.br/Restaurants-g303428-c26-Joao_Pessoa_St

In [28]:
from IPython.display import Markdown
Markdown(result.raw)

# Melhores Restaurantes Italianos em João Pessoa

João Pessoa, com sua vibrante cena gastronômica, é o lar de uma variedade de restaurantes italianos que encantam os paladares mais exigentes. Neste artigo, vamos explorar os melhores estabelecimentos que trazem o sabor irresistível da Itália, desde pratos clássicos até experiências únicas de gelato. Se você está em busca de uma experiência romântica ou um jantar em família, esses restaurantes têm tudo a oferecer.

## 1. Tartuferia Savitar: Um Toque de Sofisticação

Localizado no coração do Centro, o **Tartuferia Savitar** destaca-se pela sua atmosfera sofisticada e atendimento impecável. É o lugar perfeito para casais e amantes da gastronomia que buscam uma experiência acima da média. Os pratos, como o **risoto de trufa** e a pasta ao molho de queijo, são verdadeiras obras de arte, apresentando uma combinação de sabores que despertam os sentidos. O ambiente acolhedor e bem decorado faz com que você se sinta em um verdadeiro salão italiano, ideal para uma noite especial.

Neste restaurante, cada garfada é uma viagem pela Itália, e isso se reflete não apenas nos pratos, mas também nos ingredientes frescos utilizados. Os amantes de trufas vão se deliciar com o menuzinho tentador e a criatividade das opções oferecidas. Faça uma reserva e vivencie um momento gastronômico único!

## 2. Buongustaio Ristorante Tambaú: Autenticidade e Convivialidade

Se você procura uma autêntica experiência italiana, o **Buongustaio Ristorante Tambaú** é a escolha certa. Este restaurante é ideal para famílias e grupos que desejam partilhar uma refeição num ambiente acolhedor. Os pratos principais, como a clássica **pizza de margherita** e a saborosa **lasanha**, são preparados de forma tradicional, garantindo que cada mordida seja repleta de sabores familiares.

A atmosfera é vibrante e amigável, tornando-o um local ideal para comemorações ou jantares descontraídos. Com uma variedade de opções no cardápio, todos na mesa encontrarão algo que agrade, sem mencionar o atendimento cordial que faz todos se sentirem em casa. Não deixe de experimentar as sobremesas caseiras!

## 3. Famiglia Muccini Ristorante: Tradição em Cada Prato

O **Famiglia Muccini Ristorante**, situado em Tambaú, é um verdadeiro convite à celebração da culinária italiana. Este restaurante é especialmente popular entre famílias e casais, que podem desfrutar de uma variedade de pratos, como o **fettuccine carbonara** e a deliciosa **pizza de quatro queijos**. As porções generosas e a variedade do cardápio asseguram que todos saiam satisfeitos.

O ambiente acolhedor e decorado com um toque rústico italiano cria uma ótima atmosfera para qualquer ocasião. Os chefs se dedicam a usar ingredientes frescos e autênticos, oferecendo uma verdadeira experiência gastronômica quej entrega sabor e autenticidade a cada refeição. Uma visita aqui certamente irá agregar novos sabores ao seu repertório!

## 4. Sapore D'Italia JP: Sabor e Agilidade

Para os jovens casais que buscam uma opção rápida e deliciosa, o **Sapore D'Italia JP** em Jardim Luna é uma excelente escolha. Com um menu repleto de opções saborosas, como o **spaghetti à putanesca** e o famoso **gelato artesanal**, este restaurante combina rapidez e qualidade sem comprometer o sabor.

A atmosfera descontraída e o atendimento atencioso fazem deste local uma parada ideal para um lanche rápido ou um romântico jantar a dois. As combinações de sabores dos gelatos são especialmente elogiadas, tornando-o um local imperdível para os amantes de sobremesas. Não deixe de experimentar!

## 5. Orama Rooftop: Gastronomia com Vista

Para uma experiência gastronômica verdadeiramente memorável, o **Orama Rooftop** no Cabo Branco oferece uma vista panorâmica deslumbrante da cidade. É o lugar perfeito para casais e visitantes que desejam um jantar romântico com um toque especial. Os pratos, como o **risoto de cogumelos** e o clássico **tiramisu**, combinam sabor e apresentação, criando momentos inesquecíveis.

A atmosfera é perfeita para contemplar o pôr do sol enquanto saboreia uma refeição italiana de alta qualidade. Os ambientes românticos são ideais para jantares a luz de velas, fazendo deste restaurante uma escolha popular para ocasiões especiais. A experiência completa, com bela vista e deliciosa comida, com certeza deixará lembranças felizes.

## 6. Zibello Gelateria: O Paraíso dos Gelatos

O **Zibello Gelateria** é uma parada obrigatória para famílias e visitantes que desejam experimentar os melhores gelatos da cidade. Com uma variedade de sabores irresistíveis, como o exótico **gelato de pistache** e o apetitoso **waffle com sorvete**, essa gelateria faz do final da sua refeição um momento de pura alegria.

Além disso, a atmosfera descontraída e o atendimento amigável fazem desta gelateria um ótimo lugar para relaxar e desfrutar de uma sobremesa após um dia explorando João Pessoa. Uma visita aqui é garantia de satisfação, especialmente para aqueles que buscam uma fuga doce e refrescante.

## 7. Al Dente Cucina: Um Toque Casual com Estilo

Localizada no Bessa, a **Al Dente Cucina** oferece um ambiente casual ideal para grupos e jovens que desejam desfrutar de pratos italianos fartos e saborosos. Os destaques do menu incluem a clássica **pizza margherita** e o **penne ao pesto**, que são preparados com cuidado e atenção aos detalhes.

Este restaurante é perfeito para uma refeição descontraída após um dia de trabalho ou atividades. Além disso, o preço acessível e a qualidade dos pratos tornam a experiência ainda mais gratificante. Torne sua noite ainda mais especial compartilhando deliciosos pratos com amigos e familiares!

## 8. Trattoria de Origem: A Experiência Gourmet

No coração de Manaira, a **Trattoria de Origem** se destaca por oferecer uma verdadeira experiência italiana com ingredientes frescos e uma proposta gastronômica diferenciada. Ideal para casais e gourmets, pratos como **gnocchi com molho de ragu** e **bisteca à Fiorentina** são apresentações que encantam os olhos e o paladar.

O ambiente requintado e a atenção ao cliente transformam cada visita em um evento especial, fazendo deste restaurante uma excelente escolha para celebrar momentos especiais. A dedicação da equipe em proporcionar uma experiência única faz você sentir como se estivesse jantando em uma trattoria típica italiana.

## 9. Sapore D'Italia: A Tradição em Diversidade

O **Sapore D'Italia** é um lugar querido por antigos clientes e novos visitantes. Com um cardápio diverso que inclui pratos como a **lasanha à bolonhesa** e um bufê de antipasto, este restaurante é ideal para quem busca experiências gastronômicas únicas e repletas de sabor.

A atmosfera acolhedora promove um ambiente amigável e familiar, perfeito para jantares em grupo ou reuniões sociais. Se você é um amante da boa comida italiana, não deixe de visitar este local e se deixar surpreender pelas delícias que ele tem a oferecer!

## 10. Nonno Cucina: O Sabor da Casa

Finalmente, o **Nonno Cucina** em Centro é o restaurante ideal para casais e famílias que desejam compartilhar refeições que evocam a dieta italiana com um toque americano. Os pratos variam de **canelone de ricota** a clássicas **pizzas de mussarela**, cada um preparado como se fosse para a própria família.

O ambiente familiar convida a momentos de alegria e união à mesa, tornando-o um local perfeito para festas e celebrações. A combinação de um cardápio diversificado e um espaço acolhedor é a receita certa para uma experiência memorável.

## Conclusão: O Melhor da Culinária Italiana em João Pessoa

João Pessoa apresenta uma rica oferta de restaurantes italianos que satisfazem todos os gostos e preferências, seja para jantares românticos, refeições em família ou uma rápida pausa para gelato. Cada um desses estabelecimentos oferece uma experiência única que valoriza a tradicional culinária italiana com um toque local. Não importa qual escolha você faça, o importante é se deliciar com o sabor e as histórias que cada prato tem para contar. Faça suas reservas e prepare-se para uma deliciosa aventura gastronômica na capital da Paraíba!